Problem Statement.

You are given an integer n representing the size of a 0-indexed memory array. All memory units are initially free.

You have a memory allocator with the following functionalities:

    Allocate a block of size consecutive free memory units and assign it the id mID.
    Free all memory units with the given id mID.

Note that:

    Multiple blocks can be allocated to the same mID.
    You should free all the memory units with mID, even if they were allocated in different blocks.

Implement the Allocator class:

    Allocator(int n) Initializes an Allocator object with a memory array of size n.
    int allocate(int size, int mID) Find the leftmost block of size consecutive free memory units and allocate it with the id mID. Return the block's first index. If such a block does not exist, return -1.
    int freeMemory(int mID) Free all memory units with the id mID. Return the number of memory units you have freed.

 

Example 1:

Input
["Allocator", "allocate", "allocate", "allocate", "freeMemory", "allocate", "allocate", "allocate", "freeMemory", "allocate", "freeMemory"]
[[10], [1, 1], [1, 2], [1, 3], [2], [3, 4], [1, 1], [1, 1], [1], [10, 2], [7]]
Output
[null, 0, 1, 2, 1, 3, 1, 6, 3, -1, 0]

Explanation
Allocator loc = new Allocator(10); // Initialize a memory array of size 10. All memory units are initially free.
loc.allocate(1, 1); // The leftmost block's first index is 0. The memory array becomes [1,_,_,_,_,_,_,_,_,_]. We return 0.
loc.allocate(1, 2); // The leftmost block's first index is 1. The memory array becomes [1,2,_,_,_,_,_,_,_,_]. We return 1.
loc.allocate(1, 3); // The leftmost block's first index is 2. The memory array becomes [1,2,3,_,_,_,_,_,_,_]. We return 2.
loc.freeMemory(2); // Free all memory units with mID 2. The memory array becomes [1,_, 3,_,_,_,_,_,_,_]. We return 1 since there is only 1 unit with mID 2.
loc.allocate(3, 4); // The leftmost block's first index is 3. The memory array becomes [1,_,3,4,4,4,_,_,_,_]. We return 3.
loc.allocate(1, 1); // The leftmost block's first index is 1. The memory array becomes [1,1,3,4,4,4,_,_,_,_]. We return 1.
loc.allocate(1, 1); // The leftmost block's first index is 6. The memory array becomes [1,1,3,4,4,4,1,_,_,_]. We return 6.
loc.freeMemory(1); // Free all memory units with mID 1. The memory array becomes [_,_,3,4,4,4,_,_,_,_]. We return 3 since there are 3 units with mID 1.
loc.allocate(10, 2); // We can not find any free block with 10 consecutive free memory units, so we return -1.
loc.freeMemory(7); // Free all memory units with mID 7. The memory array remains the same since there is no memory unit with mID 7. We return 0.

 

Constraints:

    1 <= n, size, mID <= 1000
    At most 1000 calls will be made to allocate and freeMemory.



# Doubly Linked List - O(N) runtime, O(N) space

In [7]:
from collections import defaultdict

class Node:
    def __init__(self, index: int, size: int, id: int):
        self.index = index
        self.size = size
        self.id = id # -1 = free
        self.prev = None
        self.next = None

class Allocator:

    def __init__(self, n: int):
        self.head = Node(0, n, -1)
        self.used = defaultdict(list)

    def allocate(self, size: int, mID: int) -> int:
        node = self.head
        while node:
            if node.id == -1 and node.size >= size:
                node.id = mID
                if node.size > size: 
                    nextNode = Node(node.index + size, node.size - size, -1)
                    node.size = size
                    nextNext = node.next
                    node.next = nextNode
                    nextNode.prev = node
                    nextNode.next = nextNext
                    if nextNext: nextNext.prev = nextNode
                self.used[mID].append(node)
                return node.index
            node = node.next
        return -1

    def freeMemory(self, mID: int) -> int:
        if mID not in self.used: return 0
        ret = 0
        for node in self.used[mID]:
            ret += node.size
            prevN = node.prev
            nextN = node.next
            node.id = -1
            if nextN and nextN.id == -1:
                node.size += nextN.size
                node.next = nextN.next
                if nextN.next: nextN.next.prev = node
            if prevN and prevN.id == -1:
                prevN.size += node.size
                prevN.next = node.next
                if node.next: node.next.prev = prevN
            
        self.used.pop(mID)
        return ret

In [13]:
obj = Allocator(10)
print(obj.allocate(5, 1))
print(obj.allocate(3, 2))
print(obj.freeMemory(1))

0
5
5
